In [1]:
from langchain.chat_models import ChatOpenAI
#class provides more chat-related methods, such as completion_with_retry,
#get_num_tokens_from_messages to make it more user-friendly when build chatbot related applications.
#ChatOpenAI <-- BaseChatModel <- BaseLanguageModel

from langchain.agents import create_csv_agent, create_pandas_dataframe_agent
# For creating the agents
#Agents determines which actions to take and in what order.

from langchain.agents.agent_types import AgentType
#AgentType is just a class which has key value pair of different agents and their respective models

import pandas as pd    
# for dataset related things

import openai
#for setting the openai (chat gpt) environment

import langchain 
from langchain import PromptTemplate,LLMChain
#for creating the prompts

In [1]:
import json

In [4]:
with open("api_key.json") as fp:
    key_file = json.load(fp)
    api_key = key_file.get('api_key')

In [8]:
#Set openai api key
import os
os.environ["OPENAI_API_KEY"] = api_key

In [10]:
openai.api_key = os.environ["OPENAI_API_KEY"]
MODEL = "gpt-3.5-turbo"

In [3]:
##example script to send a request with openai gpt models
# response = openai.ChatCompletion.create(
#     model=MODEL,
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "Explain a prime number in briefly in 1 line"},
#     ],
#     temperature=0,
# )

# print(response['choices'][0]['message']['content'])

A prime number is a number greater than 1 that is only divisible by 1 and itself.


In [4]:
data_file='C:\Mani\sample_data.csv'

df = pd.read_csv(data_file)

In [13]:
pd.options.display.max_columns = None
df.head(2)

,_id,availability_dt,country,crt_dt,data_source,depot_id,depot_type,drug_description,id,irt_depot_name,last_gr_date,lot_expiry,lot_no,material_desc,material_id,name,original_release_qty,source,stock_blocked,stock_in_tfr,stock_quality_insp,stock_restricted_use,stock_unrestricted,storage_loc,study_id,total_available_med_id,total_damaged_mad_id,total_dns_med_id,total_expired_med_id,total_pending_med_id,total_unavailable_med_id,uom
0,6512e00dbcff54a380813017,1970-01-01T00:00:00.000Z,Belgium,1970-01-01T00:00:00.000Z,endpoint,DD-9003,Distribution Depot,CC-92480 (ALPS-986348) CAP 0.2MG (1BTLX21) CA057,FG-50223-ACG0853-CA057-008-NA-NA-CC-92480 (ALP...,EUCW - Belgium,1970-01-01T00:00:00.000Z,2024-03-31T00:00:00.000Z,ACG0853,ALPS-986348CAP0.2MG(1BTLX21)CA057OLMUL P3,FG-50223,ALPS EU-Medica Regional Depot,0,IRT,0,0,0,0,64,NaN,AL-30005,64,0,0,0,0,0,ST
1,6512e00dbcff54a380813018,1970-01-01T00:00:00.000Z,China,1970-01-01T00:00:00.000Z,endpoint,DD-9018,Distribution Depot,DEXAMETHASONE TAB 4MG (1PK) CA057 OL MUL,FG-50421-X32215B.2A-CA057-001-NA-NA-DEXAMETHAS...,CHIN - China,1970-01-01T00:00:00.000Z,2027-04-30T00:00:00.000Z,X32215B.2A,DEXAMETH RATIOPHARM TAB4MG(1PK)CA057OLMU,FG-50421,Fisher China (Suzhou),0,IRT,0,0,0,0,200,NaN,AL-30036,200,0,0,0,0,0,ST


## Creating the openapi module

In [18]:
open_ai_module = ChatOpenAI(temperature=0, model_name=MODEL)

## Creating the CSV Agent

Different Agent Types:
- ZERO_SHOT_REACT_DESCRIPTION = "zero-shot-react-description": 
    - Agent has the knowledge of the current state only and Reasoning and Acting
- REACT_DOCSTORE = "react-docstore": 
    - Searching in the Docstore and Reasoning and acting
- SELF_ASK_WITH_SEARCH = "self-ask-with-search": 
    - It will ask the mini questions to itself
- CONVERSATIONAL_REACT_DESCRIPTION = "conversational-react-description": 
    - REACT with buffer memory
- CHAT_ZERO_SHOT_REACT_DESCRIPTION = "chat-zero-shot-react-description": 
    - ZERO_SHOT_REACT_DESCRIPTION with responses in the form of chat
- CHAT_CONVERSATIONAL_REACT_DESCRIPTION = "chat-conversational-react-description": 
    - Used mainly in chat bot 
- STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION = ("structured-chat-zero-shot-react-description") : 
    - In order to use multi inputs. Eg. In Fitness App giving sets of exercise takes the input(s): Age, profession, critiques.
- OPENAI_FUNCTIONS = "openai-functions": 
    - OpenAI models are now fine-tuned to recognize and generate function calls, making it easier to get JSON objects with function arguments. 
    - The OpenAI Functions Agent is tailored for this and aims for more reliable function calls than generic text or chat APIs
- OPENAI_MULTI_FUNCTIONS = "openai-multi-functions":
     - Imagine the OpenAI Multi-Function Agent is like doing two things at once. It can execute two functions in a single go. 
     - On the other hand, the OpenAI Functions Agent is like doing one thing first and then doing another. 
     - It handles one task, finishes it, and then moves on to the next task.

In [19]:
agent = create_csv_agent(
            open_ai_module,
            data_file,
            verbose=True,
            agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

C:\Mani\Anaconda3\lib\site-packages\langchain\agents\agent_toolkits\pandas\base.py:289: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


In [16]:
question1 = "How many drugs available in the country Belgium"

In [20]:
res = agent.run(question1)



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the country is Belgium, and then count the number of unique drugs.
Action: python_repl_ast
Action Input: df[df['country'] == 'Belgium']['drug_description'].nunique()
Observation: 2
Thought:

C:\Mani\Anaconda3\lib\site-packages\langchain\tools\python\tool.py:140: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


There are 2 drugs available in the country Belgium.
Final Answer: 2

> Finished chain.


In [21]:
question2 = "What is the total inventory available at location Germany"

In [23]:
res = agent.run(question2)



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the country is Germany, and then sum up the total_available_med_id column.

Action: python_repl_ast
Action Input: df[df['country'] == 'Germany']['total_available_med_id'].sum()
Observation: 0
Thought:

C:\Mani\Anaconda3\lib\site-packages\langchain\tools\python\tool.py:140: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


The total inventory available at location Germany is 0.
Final Answer: 0

> Finished chain.
